In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os

path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"
path_mask = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/src/mask_france_boolean.nc"
output_filename = "donnees_carte_35ans_journalier.nc"

print("Démarrage de la création du NetCDF journalier...")

try:
    fichiers = sorted(glob.glob(path_data))
    fichiers_select = fichiers[:420]
    print(f"Fichiers sélectionnés : {len(fichiers_select)}")

    mask = xr.open_dataset(path_mask)
    datasets_journaliers = []

    for i, fichier in enumerate(fichiers_select):
        print(f"Traitement fichier {i+1}/{len(fichiers_select)}...", end=" ")

        try:
            ds = xr.open_dataset(fichier, engine='netcdf4')

            if 'valid_time' in ds.coords:
                ds = ds.rename({'valid_time': 'time'})

            ds_masked = ds.where(mask['mask'] == 1)
            ds_masked['t2m'] = ds_masked['t2m'] - 273.15

            ds_daily = ds_masked.resample(time='1D').mean(skipna=True)

            if 't2m' in ds_daily:
                ds_daily['t2m'] = ds_daily['t2m'].astype('float32')

            datasets_journaliers.append(ds_daily)
            ds.close()
            print("Ok")

        except Exception as e_file:
            print(f"Erreur sur {fichier} : {e_file}")

    ds_final = xr.concat(datasets_journaliers, dim='time')
    ds_final = ds_final[['t2m']]
    ds_final = ds_final.rename({'t2m': 'Temperature_C'})

    encoding = {'Temperature_C': {'zlib': True, 'complevel': 5}}
    ds_final.to_netcdf(output_filename, encoding=encoding)

    print("Terminé. Fichier créé.")

except Exception as e:
    print(f"Erreur : {e}")

Démarrage de la création du NetCDF journalier...
Fichiers sélectionnés : 420
Traitement fichier 1/420... Ok
Traitement fichier 2/420... Ok
Traitement fichier 3/420... Ok
Traitement fichier 4/420... Ok
Traitement fichier 5/420... Ok
Traitement fichier 6/420... Ok
Traitement fichier 7/420... Ok
Traitement fichier 8/420... Ok
Traitement fichier 9/420... Ok
Traitement fichier 10/420... Ok
Traitement fichier 11/420... Ok
Traitement fichier 12/420... Ok
Traitement fichier 13/420... Ok
Traitement fichier 14/420... Ok
Traitement fichier 15/420... Ok
Traitement fichier 16/420... Ok
Traitement fichier 17/420... Ok
Traitement fichier 18/420... Ok
Traitement fichier 19/420... Ok
Traitement fichier 20/420... Ok
Traitement fichier 21/420... Ok
Traitement fichier 22/420... Ok
Traitement fichier 23/420... Ok
Traitement fichier 24/420... Ok
Traitement fichier 25/420... Ok
Traitement fichier 26/420... Ok
Traitement fichier 27/420... Ok
Traitement fichier 28/420... Ok
Traitement fichier 29/420... Ok
Trai